In [2]:
import csv

expected_fields = 13
count_valid_rows = 0

# ✅ Set the full path to your Drive file
input_path = '/content/drive/MyDrive/3803ict/data.csv'
output_path = '/content/drive/MyDrive/3803ict/data_fixed.csv'

with open(input_path, "r", encoding="utf-8", newline='') as f_in, open(output_path, "w", newline='', encoding="utf-8") as f_out:
    reader = csv.reader(f_in)
    writer = csv.writer(f_out)

    header = next(reader)
    writer.writerow(header)

    for row in reader:
        if len(row) == expected_fields:
            writer.writerow(row)
            count_valid_rows += 1

print("✅ Cleaned rows written:", count_valid_rows)


✅ Cleaned rows written: 318477


In [3]:
import pandas as pd

# Path to the cleaned dataset in Google Drive
csv_path = '/content/drive/MyDrive/3803ict/data_fixed.csv'

# Load using python engine and skip malformed rows (shouldn't be many after cleaning)
df = pd.read_csv(csv_path, engine="python", on_bad_lines='skip')
print("✅ Loaded rows:", len(df))

# Preview first 5 rows
df.head()


✅ Loaded rows: 318477


,Id,Title,Company,Date,Location,Area,Classification,SubClassification,Requirement,FullDescription,LowestSalary,HighestSalary,JobType
0,37404348,Casual Stock Replenisher,Aldi Stores,2018-10-07T00:00:00.000Z,Sydney,North West & Hills District,Retail & Consumer Products,Retail Assistants,Our Casual Stock Replenishers pride themselves...,NaN,0,30,NaN
1,37404337,Casual Stock Replenisher,Aldi Stores,2018-10-07T00:00:00.000Z,Richmond & Hawkesbury,NaN,Retail & Consumer Products,Retail Assistants,Our Casual Stock Replenishers pride themselves...,NaN,0,30,NaN
2,37404356,RETAIL SALES SUPERSTARS and STYLISTS Wanted - ...,LB Creative Pty Ltd,2018-10-07T00:00:00.000Z,Brisbane,CBD & Inner Suburbs,Retail & Consumer Products,Retail Assistants,BRAND NEW FLAGSHIP STORE OPENING - SUNSHINE PLAZA,NaN,0,30,NaN
3,37404330,Team member - Belrose,Anaconda Group Pty Ltd,2018-10-07T00:00:00.000Z,Gosford & Central Coast,NaN,Retail & Consumer Products,Retail Assistants,Bring it on - do you love the great outdoors a...,NaN,0,30,NaN
4,37404308,"Business Banking Contact Centre Specialist, Ni...",Commonwealth Bank - Business & Private Banking,2018-10-07T00:00:00.000Z,Sydney,Ryde & Macquarie Park,Call Centre & Customer Service,Sales - Inbound,"We are seeking highly articulate, enthusiastic...",NaN,0,30,NaN


In [4]:
# Check shape (rows, columns)
print("Shape (rows, columns):", df.shape)

# Column names and data types
df.info()

# Summary statistics for numerical columns
print("Summary statistics:")
print(df.describe())

# Check for missing values
print("Missing values per column:")
print(df.isnull().sum())

# List all columns
print("All columns:", df.columns.tolist())


Shape (rows, columns): (318477, 13)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 318477 entries, 0 to 318476
Data columns (total 13 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   Id                 318477 non-null  object
 1   Title              318477 non-null  object
 2   Company            306473 non-null  object
 3   Date               318477 non-null  object
 4   Location           197229 non-null  object
 5   Area               122658 non-null  object
 6   Classification     197229 non-null  object
 7   SubClassification  197229 non-null  object
 8   Requirement        318470 non-null  object
 9   FullDescription    302302 non-null  object
 10  LowestSalary       318477 non-null  int64 
 11  HighestSalary      318477 non-null  int64 
 12  JobType            302379 non-null  object
dtypes: int64(2), object(11)
memory usage: 31.6+ MB
Summary statistics:
        LowestSalary  HighestSalary
count  318477.000000  318

In [5]:
# Category distribution: Job type
if 'JobType' in df.columns:
    print("JobType distribution:")
    print(df['JobType'].value_counts())

# Category distribution: Classification
if 'Classification' in df.columns:
    print("\nClassification distribution:")
    print(df['Classification'].value_counts())

# Category distribution: SubClassification
if 'SubClassification' in df.columns:
    print("\nSubClassification distribution:")
    print(df['SubClassification'].value_counts())


JobType distribution:
JobType
Full Time          201225
Contract/Temp       53906
Casual/Vacation     25355
Part Time           21893
Name: count, dtype: int64

Classification distribution:
Classification
Information & Communication Technology    22796
Trades & Services                         17976
Healthcare & Medical                      17085
Hospitality & Tourism                     15523
Manufacturing, Transport & Logistics      12376
Administration & Office Support           10020
Accounting                                 9368
Education & Training                       9280
Construction                               8372
Sales                                      8119
Retail & Consumer Products                 8041
Government & Defence                       7443
Engineering                                6458
Mining, Resources & Energy                 6288
Banking & Financial Services               4633
Community Services & Development           4485
Legal                      

In [11]:
# 3. Handling fields with excessive missing data
# Fields: Location, Area, Classification, SubClassification
# These fields have excessive missing values (over 100,000 missing entries each)
# Therefore, they will not be used for correlation or prediction
# They will be retained only for descriptive statistics (such as distribution by area or job type)

excessive_missing_cols = ['Location', 'Area', 'Classification', 'SubClassification']

# Check the number of missing values for these columns
print(df[excessive_missing_cols].isnull().sum())

# Create dataframe only with these descriptive fields
df_descriptive_only = df[excessive_missing_cols]

# Example: descriptive statistics
print(df_descriptive_only.describe(include='all'))

Location             121248
Area                 195819
Classification       121248
SubClassification    121248
dtype: int64
       Location                 Area                          Classification  \
count    197229               122658                                  197229   
unique       65                   19                                      30   
top      Sydney  CBD & Inner Suburbs  Information & Communication Technology   
freq      60765                32170                                   22796   

       SubClassification  
count             197229  
unique               338  
top                Other  
freq               11349  


In [10]:
# 4. Retaining key features for core analysis
# Selected core features: Title, Company, Requirement, LowestSalary, HighestSalary
# These features are important for the main prediction or analysis tasks

key_features = ['Title', 'Company', 'Requirement', 'LowestSalary', 'HighestSalary']

# Create dataframe with only the key features
df_key_features = df[key_features]

# Display the first few rows to verify
print(df_key_features.head())

                                               Title  \
0                           Casual Stock Replenisher   
1                           Casual Stock Replenisher   
2  RETAIL SALES SUPERSTARS and STYLISTS Wanted - ...   
3                              Team member - Belrose   
4  Business Banking Contact Centre Specialist, Ni...   

                                          Company  \
0                                     Aldi Stores   
1                                     Aldi Stores   
2                             LB Creative Pty Ltd   
3                          Anaconda Group Pty Ltd   
4  Commonwealth Bank - Business & Private Banking   

                                         Requirement  LowestSalary  \
0  Our Casual Stock Replenishers pride themselves...             0   
1  Our Casual Stock Replenishers pride themselves...             0   
2  BRAND NEW FLAGSHIP STORE OPENING - SUNSHINE PLAZA             0   
3  Bring it on - do you love the great outdoors a...            

In [9]:
# 5. Excluding or using fields only for descriptive analysis
# Fields: Location, Area, Classification, SubClassification, FullDescription, JobType
# These fields are excluded from core analysis and used only for descriptive purposes

excluded_fields = ['Location', 'Area', 'Classification', 'SubClassification', 'FullDescription', 'JobType']

# Create dataframe with only the excluded/descriptive fields
df_excluded = df[excluded_fields]

# Example: descriptive statistics for the excluded fields
print(df_excluded.describe(include='all'))

       Location                 Area                          Classification  \
count    197229               122658                                  197229   
unique       65                   19                                      30   
top      Sydney  CBD & Inner Suburbs  Information & Communication Technology   
freq      60765                32170                                   22796   

       SubClassification                                    FullDescription  \
count             197229                                             302302   
unique               338                                             250901   
top                Other  <p><strong>At ALDI, our people are the key to ...   
freq               11349                                                212   

          JobType  
count      302379  
unique          4  
top     Full Time  
freq       201225  
